In [8]:
import pandas as pd

df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_data_40k.csv')
df = df[:1000]
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_demo.csv',index=False)


In [48]:
df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_data_5k.csv')
df = df[:125]
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/val_demo.csv',index=False)

In [15]:
import yaml
from easydict import EasyDict

with open('/hpc2hdd/home/yli106/smiles2mol/config/qm9_default.yml', 'r') as file:
    config = yaml.safe_load(file)

config = EasyDict(config)

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import argparse
from easydict import EasyDict
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("/hpc2hdd/home/yli106/smiles_3d/llama2_13b", trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token # Use the EOS token to pad shorter sequences
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

max_len = 0
for i in tqdm(range(len(df))):
    input_text = df['mol_block'][i]
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    if len(input_ids[0]) > max_len:
        max_len = len(input_ids[0])
max_len   

100%|██████████| 24068/24068 [01:15<00:00, 317.69it/s]


1739

In [6]:
smiles = df['canonicalize_smiles'][10]
system_prompt = 'Below is a SMILES of a molecule, generate its 3D structure. The molecule has 15 atoms and 15 bonds.'
inst = '<s>[INST] <<SYS>>\n' + system_prompt + '\n<</SYS>>\n\n' + smiles + ' [/INST] '+' </s>'
input_ids = tokenizer.encode(inst, return_tensors='pt')
len(input_ids[0])

105

In [1]:
import pickle

with open('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/test_data_200.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [1]:
import pandas as pd

df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/test_data_200.csv')

In [3]:
print(df['mol_block'][1000])


     RDKit          3D

 22 22  0  0  0  0  0  0  0  0999 V2000
   -2.8596   -0.5041    0.9348 C   0  0  0  0  0  0  0  0  0  0  0  0
   -2.4971    0.2893   -0.2109 N   0  0  0  0  0  0  0  0  0  0  0  0
   -1.7674    1.5159   -0.0341 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.1387    0.3437   -0.6870 C   0  0  1  0  0  0  0  0  0  0  0  0
    0.0167   -0.3815   -0.0217 C   0  0  2  0  0  0  0  0  0  0  0  0
    1.3329    0.3414   -0.3476 C   0  0  0  0  0  0  0  0  0  0  0  0
    2.4986   -0.1404    0.5203 C   0  0  0  0  0  0  0  0  0  0  0  0
    3.7169    0.4716    0.1787 O   0  0  0  0  0  0  0  0  0  0  0  0
    3.6131    1.4282    0.2342 H   0  0  0  0  0  0  0  0  0  0  0  0
    2.6753   -1.2105    0.3795 H   0  0  0  0  0  0  0  0  0  0  0  0
    2.2647    0.0338    1.5828 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.1906    1.4142   -0.1927 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.5744    0.1770   -1.3996 H   0  0  0  0  0  0  0  0  0  0  0  0
   -0.1206   -0.3811    1

In [1]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def preprocess(text):
    encoding = tokenizer.encode_plus(text, truncation=True, padding=True, return_tensors='pt')
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    return input_ids, attention_mask


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/hpc2hdd/home/yli106/smiles_3d/gpt2")


In [12]:
import pandas as pd
from datasets import Dataset 
df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_demo.csv')
train_dataset = Dataset.from_pandas(df[['text']])
test_dataset = train_dataset
train_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

model = AutoModelForCausalLM.from_pretrained(
    "/hpc2hdd/home/yli106/smiles_3d/llama2_7b_chat",
    # quantization_config=bnb_config,
    device_map= "auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
import pandas as pd

df = pd.read_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_data_40k.csv')
df

,smiles,canonicalize_smiles,num_atom,num_bond,mol_block,text
0,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
1,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
2,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
3,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
4,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
...,...,...,...,...,...,...
199995,N[C@H]1COCOC1=O,[H]N([H])[C@]1([H])C(=O)OC([H])([H])OC1([H])[H],15,15,\n RDKit 3D\n\n 15 15 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
199996,N[C@H]1COCOC1=O,[H]N([H])[C@]1([H])C(=O)OC([H])([H])OC1([H])[H],15,15,\n RDKit 3D\n\n 15 15 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
199997,N[C@H]1COCOC1=O,[H]N([H])[C@]1([H])C(=O)OC([H])([H])OC1([H])[H],15,15,\n RDKit 3D\n\n 15 15 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...
199998,N[C@H]1COCOC1=O,[H]N([H])[C@]1([H])C(=O)OC([H])([H])OC1([H])[H],15,15,\n RDKit 3D\n\n 15 15 0 0 0 0...,<s>[INST] <<SYS>>\nBelow is a SMILES of a mole...


In [2]:
def process_inst(smiles, num_atom, num_bond):
    system_prompt = f'Below is a SMILES of a molecule, generate its 3D structure. The molecule has {num_atom} atoms and {num_bond} bonds.'
    inst = '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n' + system_prompt + '<|eot_id|><|start_header_id|>user<|end_header_id|>\n' + smiles + '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n'
    
    return inst

In [7]:
text_col = []
for _, row in df.iterrows():
    num_atom = row['num_atom']
    num_bond = row['num_bond']
    smiles = row['canonicalize_smiles'] 
    inst = process_inst(smiles, num_atom, num_bond)
    model_answer = row['mol_block']
    text = inst + model_answer + '<|end_of_text|>'
    text_col.append(text)

df.loc[:, 'text'] = text_col

In [8]:
df

,smiles,canonicalize_smiles,num_atom,num_bond,mol_block,text
0,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...
1,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...
2,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...
3,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...
4,C[C@@]1(N2CC2)C[C@H]1O,[H]O[C@]1([H])C([H])([H])[C@]1(N1C([H])([H])C1...,19,20,\n RDKit 3D\n\n 19 20 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...
...,...,...,...,...,...,...
199995,N[C@H]1COCOC1=O,[H]N([H])[C@]1([H])C(=O)OC([H])([H])OC1([H])[H],15,15,\n RDKit 3D\n\n 15 15 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...
199996,N[C@H]1COCOC1=O,[H]N([H])[C@]1([H])C(=O)OC([H])([H])OC1([H])[H],15,15,\n RDKit 3D\n\n 15 15 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...
199997,N[C@H]1COCOC1=O,[H]N([H])[C@]1([H])C(=O)OC([H])([H])OC1([H])[H],15,15,\n RDKit 3D\n\n 15 15 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...
199998,N[C@H]1COCOC1=O,[H]N([H])[C@]1([H])C(=O)OC([H])([H])OC1([H])[H],15,15,\n RDKit 3D\n\n 15 15 0 0 0 0...,<|begin_of_text|><|start_header_id|>system<|en...


In [5]:
df.to_csv('/hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed/train_data_40k_llama3.csv', index=False)

In [1]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    HfArgumentParser,
    TrainingArguments,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import re
from tqdm import tqdm
import pandas as pd
import pickle
from conf3d import dataset
import yaml
from easydict import EasyDict

with open('/hpc2hdd/home/yli106/smiles2mol/config/qm9_default.yml', 'r') as f:
    config = yaml.safe_load(f)
config = EasyDict(config)
config.training_arguments.learning_rate = float(config.training_arguments.learning_rate)

In [2]:
# load the tokenizer
model_path = os.path.join(config.model.base_path, '%s_%s' % (config.model.type, config.model.size))
peft_path = config.model.peft_path
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token # Use the EOS token to pad shorter sequences
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# model set up
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map= "auto"
)
model = PeftModel.from_pretrained(model, peft_path)
model = model.merge_and_unload()

# load the dataset
load_path = os.path.join(config.data.base_path, '%s_processed' % config.data.dataset)
print('loading data from %s' % load_path)
with open(os.path.join(load_path, config.data.test_set), 'rb') as f:
    test_data = pickle.load(f)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/hpc2hdd/home/yli106/.conda/envs/dsaa6800/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/hpc2hdd/home/yli106/.conda/envs/dsaa6800/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/hpc2hdd/home/

loading data from /hpc2hdd/home/yli106/smiles2mol/GEOM/qm9_processed


In [3]:
num_conf = len(test_data[0][3])
input_text = dataset.process_inst(test_data[0][0], test_data[0][1], test_data[0][2])
print(input_text)
input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids = input_ids.to('cuda')

<s>[INST] <<SYS>>
Below is a SMILES of a molecule, generate its 3D structure. The molecule has 19 atoms and 18 bonds.
<</SYS>>

[H]C#CC(=O)[C@]([H])(O[H])C([H])([H])C([H])([H])C([H])([H])[H] [/INST] 


In [4]:
# define StoppingCriteria
class CustomStoppingCriteria(StoppingCriteria):
    def __call__(self, input_ids, scores):
        # check if the end signiture is generated
        if 11056 in input_ids[0]:
            return True 
        # check if the number of line breaks reaches certain length
        elif (input_ids[0] == 13).sum().item() >= test_data[0][1]+test_data[0][2]+10:
            return True
        else:
            return False

In [5]:
from torch.cuda import empty_cache

generated_texts = []
batch_size= 5
for _ in range((config.inference.multiplier*num_conf) // batch_size):
    output_sequences = model.generate(
        input_ids=input_ids, 
        max_length=config.inference.max_length, 
        do_sample=config.inference.do_sample, 
        top_k=config.inference.top_k, 
        top_p=config.inference.top_p, 
        temperature=config.inference.temperature, 
        eos_token_id=tokenizer.eos_token_id, 
        stopping_criteria=[CustomStoppingCriteria()], 
        num_return_sequences=batch_size
    )
    generated_texts += [tokenizer.decode(sequence, skip_special_tokens=True) for sequence in output_sequences]
    empty_cache()  # 清除显存缓存

# 如果num_conf不是batch_size的整数倍，处理剩余的序列
remainder = (config.inference.multiplier*num_conf) // batch_size
if remainder > 0:
    output_sequences = model.generate(
        input_ids=input_ids, 
        max_length=config.inference.max_length, 
        do_sample=config.inference.do_sample, 
        top_k=config.inference.top_k, 
        top_p=config.inference.top_p, 
        temperature=config.inference.temperature, 
        eos_token_id=tokenizer.eos_token_id, 
        stopping_criteria=[CustomStoppingCriteria()], 
        num_return_sequences=remainder
    )
    generated_texts += [tokenizer.decode(sequence, skip_special_tokens=True) for sequence in output_sequences]

In [11]:
from conf3d import dataset

processed_texts = []
for i in range(len(generated_texts)):
    test_text = dataset.get_mol_block(generated_texts[i], test_data[0][0], test_data[0][1], test_data[0][2])
    substring = test_text.split('END')[0]+'END'
    processed_texts.append(substring)

In [19]:
from rdkit import Chem
from rdkit.Chem import AllChem
import re
from rdkit.Chem import rdDetermineBonds
from rdkit.Chem.rdmolops import RemoveHs
from rdkit.Chem import rdMolAlign as MA
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

from conf3d import dataset

gen_data = []
for i in range(len(generated_texts)):
    with open('test.mol', 'w') as f:
        f.write(processed_texts[i])
    try:
        gen_mol = Chem.MolFromMolFile('/hpc2hdd/home/yli106/smiles2mol/test.mol')
        gen_mol = RemoveHs(gen_mol)
        gen_data.append(gen_mol)
    except:
        pass


[15:22:48] Cannot convert '-3.' to int on line 24
[15:22:48] Explicit valence for atom # 8 H, 2, is greater than permitted
[15:22:48] Cannot convert '-4.' to int on line 24
[15:22:48] Atom line too short: '  1  2  1  0' on line 23
[15:22:48] Atom line too short: '  1  2  1  0' on line 23
[15:22:48] Atom line too short: '  1  2  1  0' on line 23
[15:22:48] Atom line too short: '  1  2  1  0' on line 23
[15:22:48] EOF hit while reading bonds
[15:22:48] EOF hit while reading bonds
[15:22:48] Cannot convert 'END' to int on line 42
[15:22:48] Atom line too short: '  1  2  1  0' on line 23
[15:22:48] EOF hit while reading bonds
[15:22:48] Atom line too short: '  1  2  1  0' on line 23
[15:22:48] EOF hit while reading bonds
[15:22:48] EOF hit while reading bonds
[15:22:48] Atom line too short: '  1  2  1  0' on line 23
[15:22:48] EOF hit while reading bonds
[15:22:48] Atom line too short: '  1  2  1  0' on line 23
[15:22:48] EOF hit while reading bonds
[15:22:48] Atom line too short: '  1  2 

In [22]:
success = 0
valid = 0
for gen_mol in gen_data:
    min_rmsd = 1000
    for raw_ref_mol in test_data[0][3]:
        ref_mol = RemoveHs(raw_ref_mol) 
        if Chem.MolToSmiles(gen_mol) == Chem.MolToSmiles(ref_mol):
            rmsd = MA.GetBestRMS(gen_mol, ref_mol)
            success+=1
            if rmsd<min_rmsd:
                min_rmsd = rmsd
    if min_rmsd<=0.5:
        valid+=1
        # print(Chem.MolToSmiles(ref_mol))
        # print(Chem.MolToMolBlock(ref_mol))

In [23]:
min_rmsd

0.2669939154386034